In [29]:
import pickle
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from tqdm import tqdm

In [14]:
tokId2groupId = pickle.load(open("gr-t5-base-emb-p1-5-only-first/tokId_tokGroupId.pickle", "rb"))
groupId2tokIdList = pickle.load(open("gr-t5-base-emb-p1-5-only-first/tokGroupId_tokIdList.pickle", "rb"))
contextualized_file = pickle.load(open("gr-t5-base-emb-p1-5-only-first/tokId_emb.pickle", "rb"))
tokId2tokText = pickle.load(open("gr-t5-base-emb-p1-5-only-first/tokId_tokText.pickle", "rb"))

In [15]:
lm_head = torch.tensor(np.array(list(contextualized_file.values())))

In [16]:
count_tokId_per_groupId = []
for groupId, tokId_list in groupId2tokIdList.items():
    count_tokId_per_groupId.append(len(tokId_list))
    
count_tokId_per_groupId= np.array(count_tokId_per_groupId)
count_tokId_per_groupId.min(), count_tokId_per_groupId.max(), count_tokId_per_groupId.mean(), np.median(count_tokId_per_groupId)

(1, 8210, 16.39918809201624, 4.0)

In [18]:
len(groupId2tokIdList)

12563

In [19]:
values_list = []
gt_not_1st_list = []
total_cnt = 0
prob_1st = []
under_09 = []
for groupId in tqdm(range(min(1000,len(groupId2tokIdList)))):
    tokId_list = groupId2tokIdList[groupId]
    embs = lm_head[tokId_list]

    score = torch.inner(embs, embs)
    topk = torch.topk(score, len(tokId_list))
    values = topk.values
    indices = topk.indices

    inner_scores = values.clone()
    values = nn.functional.softmax(values, dim=-1)
    indices = torch.tensor(tokId_list)[indices]
    
    for index, tokId in zip(indices[:,0], groupId2tokIdList[groupId]):
        total_cnt += 1
        if index != tokId:
            gt_not_1st_list.append(tokId)
            
    prob_1st.extend(values[:,0].tolist())
    for prob, inner_score in zip(values[:,0].tolist(), inner_scores[:,0].tolist()):
        under_09.append((prob, inner_score, tokId2tokText[tokId_list[0]]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:18<00:00, 12.77it/s]


In [20]:
total_cnt, len(gt_not_1st_list), len(gt_not_1st_list)/total_cnt

(117928, 1542, 0.013075775049182552)

In [22]:
prob_1st = np.array(prob_1st)
len(prob_1st), prob_1st.min(), prob_1st.max(), prob_1st.mean(), np.median(prob_1st)

(117928, 0.0003197461483068764, 1.0, 0.9726794669315938, 1.0)

In [26]:
(prob_1st < 0.9).sum() / len(prob_1st)

0.05524557357031409

In [278]:
sorted(under_09)[-10:]

[(73.57726287841797, 0.8815833926200867, '▁('),
 (73.75415802001953, 0.8425592184066772, 'season'),
 (73.78164672851562, 0.569462239742279, '▁('),
 (74.30223083496094, 0.7905899882316589, 'season'),
 (74.73615264892578, 0.6935755610466003, '▁('),
 (75.08045959472656, 0.8994543552398682, 'season'),
 (75.16217041015625, 0.7521727681159973, '▁('),
 (75.27179718017578, 0.8548920154571533, '▁('),
 (75.29521179199219, 0.8719983696937561, '▁('),
 (77.0504379272461, 0.8278325200080872, '▁(')]

In [27]:
text_set = set()
score_text_list = []
score_list = []
for prob, score, text in sorted(under_09):
    score_list.append(score)
    if prob < 0.3 and score >= 50:
        print((prob,score,text))
        text_set.add(text)
text_set

(0.0612228661775589, 56.18656539916992, '▁series')
(0.07518958300352097, 52.79911422729492, 'al')
(0.08377470821142197, 60.16774368286133, '▁series')
(0.08551058918237686, 53.761871337890625, 'al')
(0.10053066909313202, 52.95563888549805, 'al')
(0.10303737223148346, 53.94261169433594, 'al')
(0.11063974350690842, 66.23710632324219, '▁film')
(0.11201302707195282, 71.63798522949219, '▁election')
(0.11280199885368347, 65.57821655273438, '▁film')
(0.11789960414171219, 63.755775451660156, '▁film')
(0.12236788123846054, 72.12886810302734, '▁election')
(0.12783844769001007, 50.43895721435547, 'n')
(0.1296350657939911, 63.217689514160156, '▁Top')
(0.1329270899295807, 72.10143280029297, '▁election')
(0.13925637304782867, 66.26937103271484, '▁Top')
(0.14082962274551392, 55.11247253417969, 'al')
(0.1420569121837616, 55.02806091308594, 'al')
(0.1430654376745224, 72.51897430419922, '▁election')
(0.14312593638896942, 71.80255126953125, '▁election')
(0.1431724727153778, 72.08014678955078, '▁election')

{"'",
 ')',
 'I',
 'a',
 'al',
 'and',
 'film',
 'g',
 'l',
 'n',
 'or',
 're',
 's',
 'season',
 'y',
 '–',
 '▁',
 '▁(',
 '▁100',
 '▁America',
 '▁An',
 '▁Area',
 '▁Division',
 '▁Fact',
 '▁Final',
 '▁Great',
 '▁Law',
 '▁League',
 '▁Men',
 '▁Model',
 '▁Next',
 '▁Off',
 '▁Olympics',
 '▁Single',
 '▁Star',
 '▁States',
 '▁The',
 '▁Top',
 '▁United',
 '▁and',
 '▁election',
 '▁film',
 '▁presidential',
 '▁season',
 '▁series',
 '▁the'}

In [28]:
score_list = np.array(score_list)
score_list.min(), score_list.max(), score_list.mean(), np.median(score_list)

(0.900947093963623, 96.7463607788086, 64.62101938855466, 64.92513656616211)

In [244]:
groupId = 3000
tokId_list = groupId2tokIdList[groupId]
embs = lm_head[tokId_list]

score = torch.inner(embs, embs)
topk = torch.topk(score, len(tokId_list))
values = topk.values
indices = topk.indices

#values = nn.functional.softmax(values, dim=-1)
indices = torch.tensor(tokId_list)[indices]
values.shape, indices.shape, tokId2tokText[tokId_list[0]]

(torch.Size([11, 11]), torch.Size([11, 11]), 'agh')

In [245]:
for index, tokId in zip(indices[:,0], groupId2tokIdList[groupId]):
    assert index == tokId

In [246]:
nn.functional.softmax(values, dim=-1)[:,0].tolist()

[0.9999998807907104,
 0.9999985694885254,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999998807907104,
 1.0,
 1.0,
 1.0]

In [249]:
indices

tensor([[  8248, 167842,  12250,  67071,  14174, 190005, 179245,  51136, 174338,
         138904,  99992],
        [ 12250, 167842,   8248,  67071, 174338,  14174, 179245, 138904, 190005,
          51136,  99992],
        [ 14174,   8248, 167842, 179245,  67071, 190005, 138904, 174338,  51136,
          12250,  99992],
        [ 51136, 190005, 174338,  99992, 179245,   8248,  14174,  67071, 138904,
         167842,  12250],
        [ 67071,   8248, 190005, 167842, 179245,  14174,  12250,  51136, 174338,
          99992, 138904],
        [ 99992, 138904,  51136, 190005,  14174, 174338, 179245,   8248, 167842,
          67071,  12250],
        [138904,  99992, 174338,  14174, 190005,  51136,   8248, 167842,  12250,
         179245,  67071],
        [167842,  12250,   8248,  67071,  14174, 174338, 190005, 179245, 138904,
          51136,  99992],
        [174338, 190005,  51136, 138904,  14174,  12250,   8248,  99992, 167842,
         179245,  67071],
        [179245,  67071,  51136,  141

In [34]:
df_org = pd.read_csv("nq_toy_corpus_p1-5.csv")
df_scale = pd.read_csv("scale_3_kilt_corpus_p1-5.csv")

In [49]:
for corpusId in range(len(df_org)):
    title1 = df_org["corpus"][corpusId]
    title2 = df_scale["corpus"][corpusId]
    assert title1 == title2
    
    context1 = df_org["context"][corpusId]
    context2 = df_scale["context"][corpusId]
    if context1 != context2:
        print(context1, context2)

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
